In [320]:
#解析生成json的内容
#解析决策过程的日志
import os
import json
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def paese_result_json(log_path):
    #读取json文件
    res=json.load(open(log_path,"r"))['update']
    #去除掉init_state部分
    static={}
    
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        
        for action in actions:
            if str(action['class']) + '_' + str(action['id']) not in static:
                static[str(action['class']) + '_' + str(action['id'])]={}
            static[str(action['class']) + '_' + str(action['id'])][action['action_type']]=static[str(action['class']) + '_' + str(action['id'])][action['action_type']]+1 if action['action_type'] in static[str(action['class']) + '_' + str(action['id'])] else 1
            if 'damage' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['damage']=[]
            if 'damage' in action:
                for damage in action['damage']:
                    static[str(action['class']) + '_' + str(action['id'])]['damage'].append(damage[1][0])
                    
                    
        for state in states:
            if len(state)>0:
                for change in state:
                    if change[1][2]=="Hp":
                        #pass
                        if str(change[1][0]) + '_' + str(change[1][1]) not in static:
                            static[str(change[1][0]) + '_' + str(change[1][1])]={}
                        
                        if '最终HP' not in static[str(change[1][0]) + '_' + str(change[1][1])]:
                            static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=''
                        static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=change[2][1]
            
    for k in static.keys():
        static[k]['总伤害']=sum(static[k]['damage'])
        static[k]['总伤害次数']=len(static[k]['damage'])
        static[k]['平均伤害']=sum(static[k]['damage'])/len(static[k]['damage']) if len(static[k]['damage'])>0 else 0
        if 'RIGHT' not in static[k]:
            static[k]['RIGHT']=0
        if 'LEFT' not in static[k]:
            static[k]['LEFT']=0
        if 'TOP' not in static[k]:
            static[k]['TOP']=0
        if 'BOTTOM' not in static[k]:
            static[k]['BOTTOM']=0
        static[k]['总移动次数']=static[k]['RIGHT']+static[k]['LEFT']+static[k]['TOP']+static[k]['BOTTOM']


        #隐藏具体伤害
        #static[k].pop('damage')
        #按照key 排序
        static[k]=dict(sorted(static[k].items(),key=lambda x:x[0]))
    return static


In [321]:
log_files_path = os.getcwd()+"/test_data"
    #读取这个路径下所有文件
log_files = os.listdir(log_files_path)
for log_file in log_files:
    if log_file.endswith(".json"):
        log_path = log_files_path+"/"+log_file
        static_res_json=paese_result_json(log_path)
        df=pd.DataFrame(static_res_json).T

        print(log_file)
        display(df)

for_qiangye_skill_82.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,最终HP
monster_18714666,3,2,2,3,2,9,"[100.0, 100.0, 100.0]",100.0,300.0,3,9,300.0
hero_5002,1,0,15,NaN,16,1,[],0,0,0,32,119.0


for_qiangye_skill_81.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,最终HP
monster_18714666,3,2,2,3,2,9,"[113.25, 113.25, 127.0]",117.833333,353.5,3,9,NaN
hero_5002,1,0,15,NaN,16,1,[],0,0,0,32,146.5


for_qiangye_skill_79.json


,BOTTOM,LEFT,RIGHT,SKILL_77,TOP,WAIT,damage,平均伤害,总伤害,总伤害次数,总移动次数,最终HP
monster_18714666,3,2,2,3,2,9,"[127.0, 127.0, 127.0]",127.0,381.0,3,9,NaN
hero_5002,1,0,15,NaN,16,1,[],0,0,0,32,119.0


In [322]:
def parse_result_json_detail(log_path):
    res = json.load(open(log_path, "r"))['update']
    # 去除掉init_state部分
    static = {}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        for i in range(len(actions)):
            #if actions[i]['action_type']=='SKILL_104':
                print(actions[i],states[i])
log_files_path = os.getcwd() + "/test_data"
    # 读取这个路径下所有文件
log_path = log_files_path + "/for_qiangye_skill_"+str(82)+".json"
parse_result_json_detail(log_path)

{'action_type': 'TOP', 'move_position': [5, 1, 4], 'id': 18714666, 'class': 'monster'} [['change', ['map', 0, 32, 'Block'], [2, 1]], ['change', ['map', 0, 33, 'Block'], [1, 2]], ['change', ['monster', 18714666, 'position', 2], [3, 4]]]
{'action_type': 'LEFT', 'move_position': [4, 1, 4], 'id': 18714666, 'class': 'monster'} [['change', ['map', 0, 16, 'Block'], [1, 2]], ['change', ['map', 0, 33, 'Block'], [2, 1]], ['change', ['monster', 18714666, 'position', 0], [5, 4]]]
{'action_type': 'SKILL_77', 'atk_range': [[4, 1, 3]], 'atk_position': [4, 1, 3], 'release_range': [[3, 1, 4], [4, 1, 3], [4, 1, 5], [5, 1, 4], [4, 1, 4]], 'damage': [[['hero', 5002], [100.0], [100.0], ['Equal']]], 'id': 18714666, 'class': 'monster'} [['change', ['hero', 5002, 'Hp'], [500, 373.0]], ['change', ['monster', 18714666, 'Hp'], [600, 500.0]]]
{'action_type': 'WAIT', 'id': 5002, 'class': 'hero'} []
{'action_type': 'RIGHT', 'move_position': [5, 1, 4], 'id': 18714666, 'class': 'monster'} [['change', ['map', 0, 16, '

In [319]:
def parse_decision_log(log_path):
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                if v['stepname']=='evaluate执行判断':
                   if '判断是否满足恢复判定' in v['selection'] and v['result_len']=='True':
                        print(k,v)

In [289]:

tmp_log_file = os.getcwd()+"/../log/logs.json"
parse_decision_log(tmp_log_file)

2024-09-23 15:43:44.018736_24 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定_<function lambda_is_need_to_healing.<locals>.<lambda> at 0x7f80f0e25a60>', 'result_len': 'True'}
2024-09-23 15:43:45.924549_64 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定_<function lambda_is_need_to_healing.<locals>.<lambda> at 0x7f80f0d67598>', 'result_len': 'True'}
2024-09-23 15:43:48.346668_122 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定_<function lambda_is_need_to_healing.<locals>.<lambda> at 0x7f80f0e25b70>', 'result_len': 'True'}
2024-09-23 15:43:50.995422_162 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定_<function lambda_is_need_to_healing.<locals>.<lambda> at 0x7f80f0d676a8>', 'result_len': 'True'}
2024-09-23 15:43:54.225000_220 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定_<function lambda_is_need_to_healing.<locals>.<lambda> at 0x7f80a8118ae8>', 'result_len': 'True'}
2024-09-23 15:43:57.005641_260 {'stepname': 'evaluate执行判断', 'selection': '判断是否满足恢复判定_<functi